# Importar Paquetes

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
#Para lectura, escritura y guardado de video
from IPython.display import HTML
from moviepy.editor import VideoFileClip
%matplotlib inline

# Funciones

In [2]:
#Promediando y extrapolando las lineas 
def average_slope_intercept(lines):
    """
    Encuentra la pendiente y la intercepcion de las lineas izquierda y derecha de la imagen, devuelve el arreglo de la linea
    izquierda y derecha
    Parametros: 
     lines: La salida de la transformada de Hough 
    """
    """
    Find the slope and intercept of the left and right lanes of each image.
        Parameters:
            lines: The output lines from Hough Transform.
    """
    left_lines    = [] #(slope, intercept)
    left_weights  = [] #(length,)
    right_lines   = [] #(slope, intercept)
    right_weights = [] #(length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - (slope * x1)
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
            if slope < 0:
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    left_lane  = np.dot(left_weights,  left_lines) / np.sum(left_weights)  if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    return left_lane, right_lane

def pixel_points(y1, y2, line):
    """
    Convierte la pendiente y la intercepcion de cada linea en puntos de pixel 
        Parametros: 
            y1: y-valor del punto inicial de la linea
            y2: y-valor del punto final de la linea 
            La linea que se desea convertir
    """
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

def lane_lines(image, lines):
    """
    Crea lineas completas de los puntos de pixel 
        Parametros: 
            La imagen original 
            Las lineas de la salida de la transformada de Hough 
        Retorna los puntos de la linea izquierda y derecha 
    """
    
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = y1 * 0.6
    left_line  = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    return left_line, right_line # Retorna los puntos de la linea izquierda y derecha 

def draw_lane_lines(image, lines, color=[255, 255, 0], thickness=12):
    """
    Dibuja las lineas en la imagen de entrada.
    Draw lines onto the input image.
        Parameters:
            image: The input test image.
            lines: The output lines from Hough Transform.
            color (Default = red): Line color.
            thickness (Default = 12): Line thickness. 
    """
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    return cv2.addWeighted(image, 0.7, line_image, 0.3, 0.0)


In [3]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    # Convertir a escala de grises
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)

    # Filtro Gaussiano
    kernel_size = 13;
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Detección de bordes
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
        
            ##Delimitar la region de interes 
    # Creamos la mascara utilizando la función fillpolly
    mask = np.zeros_like(edges)   #Retorna una mascar con zeros en la parte que no es un borde 
    if len(image.shape) > 2:
        channel_count = image.shape[2]
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
    rows, cols = image.shape[:2] #Toma los  valores 
    bottom_left  = [cols * 0.1, rows * 0.95]
    top_left     = [cols * 0.4, rows * 0.6]
    bottom_right = [cols * 0.9, rows * 0.95]
    top_right    = [cols * 0.6, rows * 0.6]
        
    # Se define un poligono de cuatro lado
    imshape = image.shape
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters for large lines

    # Make a blank the same size as our image to draw on
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 300  # maximum gap in pixels between connectable line segments
    #line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

    # Dibujar las lineas detectadas
    result = np.copy(image)

    #for line in lines:
     #   for x1,y1,x2,y2 in line:
     #       cv2.line(result,(x1,y1),(x2,y2),(255,255,0),2)   
    imagenPrueba = draw_lane_lines (image, lane_lines(image,lines)) #Draw_lane_lines funcion y lane lines 
    
    return imagenPrueba

# Leer video

In [4]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4") #Extrae el video completo
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
#fl_image: Modifica el frame remplazandolo por otro 
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:07<00:00, 29.12it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

Wall time: 8.94 s


In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format('test_videos/solidWhiteRight.mp4'))

In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))